Setup

This notebook shows you how to use your trained model to initialize the keypoint labels for the unannotated images in your annotation set.

In [ ]:
import sys
!{sys.executable} -m pip install -U deepposekit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

from deepposekit.models import load_model
from deepposekit.io import DataGenerator, ImageGenerator

from os.path import expanduser

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

HOME = expanduser("~") if not IN_COLAB else '.'
HOME = HOME +'/Documents/RamanLab'

In [ ]:
models = sorted(glob.glob(HOME + '/deepposekit-data/datasets/locust/*model*.h5'))
models

In [ ]:
annotations = sorted(glob.glob(HOME + '/deepposekit-data/datasets/locust/*annotation*.h5'))
annotations

Loading Model

In [ ]:
model = load_model(HOME + '/deepposekit-data/datasets/locust/best_model_densenet.h5')

Init New Data

In [ ]:
data_generator = DataGenerator(HOME + '/deepposekit-data/datasets/locust/annotation_data_release.h5', mode='unannotated')
image_generator = ImageGenerator(data_generator)

In [ ]:
predictions = model.predict(image_generator, verbose=1)

In [ ]:
predictions.shape

In [ ]:
data_generator[:] = predictions

In [ ]:
image, keypoints = data_generator[0]

plt.figure(figsize=(5,5))
image = image[0] if image.shape[-1] is 3 else image[0, ..., 0]
cmap = None if image.shape[-1] is 3 else 'gray'
plt.imshow(image, cmap=cmap, interpolation='none')
for idx, jdx in enumerate(data_generator.graph):
    if jdx > -1:
        plt.plot(
            [keypoints[0, idx, 0], keypoints[0, jdx, 0]],
            [keypoints[0, idx, 1], keypoints[0, jdx, 1]],
            'r-'
        )
plt.scatter(keypoints[0, :, 0], keypoints[0, :, 1], c=np.arange(data_generator.keypoints_shape[0]), s=50, cmap=plt.cm.hsv, zorder=3)

plt.show()